Loading image dataset and CSV Files

In [ ]:
import os 
import pandas
import numpy 
from augmentations import augmentations
from datasets import datasets
from PIL import Image
import cv2

images = []

for fil in os.listdir("../data/raw_data/images"):
    file_dir = os.path.join("../data/raw/data/images", fil)
    images.append(file_dir)

Loading Dataset information CSV File

In [ ]:
image_loader = numpy.vectorize(pyfunc=lambda img: Image.open(img))

train_info = pandas.read_csv("../data/raw_data/information.csv")
train_info['image'] = image_loader(images)

In [ ]:
train_info['Class'].map({
    'yes': 0,
    'no': 1,
})

Converting images to specific format

In [ ]:
def to_png(img):
    success, png_data = cv2.imencode(ext='.png', img=img)
    if success:
        png_img = cv2.imdecode(png_data, cv2.IMREAD_UNCHANGED)
        return Image.fromarray(png_img)
    else:
        raise RuntimeError('Failed to convert image')

train_info['image'] = train_info['image'].apply(lambda img: to_png(img))

Measuring noise degree by using TV (Total Variation), 
SNR (Signal-to-Noise Ratio) and NLE (Noise Local Estimation) metrics

In [ ]:
from skimage.restoration import estimate_sigma, denoise_tv_chambolle

def total_variation(image):
    dx = numpy.diff(image, axis=1)
    dy = numpy.diff(image, axis=0)
    tv = numpy.sum(numpy.abs(dx)) + numpy.sum(numpy.abs(dy))
    return tv

def noise_local_estimation(img, weight):
    if gray_img.shape[0] == 3:
        gray_img = cv2.cvtColor(gray_img, cv2.COLOR_RGB2GRAY)
    tv_img = estimate_sigma(gray_img)
    tv = numpy.sum(numpy.abs(tv_img - gray_img))
    return tv

In [ ]:
images = train_info['image'].tolist()

tv_thresh = 10 
snr_thresh = 100
nle_thresh = 200

for image in images:
    tv = total_variation(image)
    snr = signal_to_noise_ratio(image)
    nle = noise_local_estimation(image)

Initializing Datasets for training and validation

In [ ]:
from sklearn.model_selection import train_test_split 

train_indices, val_indices = train_test_split(
    numpy.arrange(train_info.shape[0]), 
    test_size=0.3, 
    stratify=train_info['class'].tolist()
)

In [ ]:
train_dataset = datasets.DeepFakeClassificationDataset(
    labels=train_info['class'][train_indices],
    images=train_info['image'][train_indices],
    transforms=augmentations.get_training_augmentations()
)

validation_dataset = datasets.DeepFakeClassificationDataset(
    labels=train_info['class'][val_indices],
    images=train_info['image'][val_indices],
    transforms=augmentations.get_validation_augmentations()
)

Post processing image quality estimation using SNR and SSIM

Handling disbalance in the dataset using Class Weighting

In [ ]:
CLASS_WEIGHTS = [1, 1]

train_dataset.weights = validation_dataset.weights = CLASS_WEIGHTS

Saving datasets to local storage

In [ ]:
import pickle 

pickle.dump(obj=train_dataset, file=open("../data/augmented_data/train_dataset.pkl", mode='wb'))
pickle.dump(obj=validation_dataset, file=open("../data/augmented_data/validation_dataset.pkl", mode='wb'))